In [ ]:
# default_exp problem_types.masklm
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
# test setup
import tensorflow as tf
from m3tl.test_base import TestBase
from m3tl.input_fn import train_eval_input_fn
test_base = TestBase()
params = test_base.params

hidden_dim = params.bert_config.hidden_size

train_dataset = train_eval_input_fn(params=params)
one_batch = next(train_dataset.as_numpy_iterator())


# Masked Language Model(masklm)

This module includes neccessary part to register Masked Language Model problem type.

## Imports and utils


In [ ]:
# export
import pickle
from typing import List

import tensorflow as tf
from loguru import logger
from m3tl.base_params import BaseParams
from m3tl.problem_types.utils import (empty_tensor_handling_loss,
                                      nan_loss_handling, pad_to_shape)
from m3tl.special_tokens import PREDICT
from m3tl.utils import (LabelEncoder, gather_indexes,
                        get_label_encoder_save_path, get_phase,
                        load_transformer_tokenizer, need_make_label_encoder)
from transformers import TFSharedEmbeddings


## Top Layer

In [ ]:
#export
class MaskLM(tf.keras.Model):
    """Multimodal MLM top layer.
    """

    def __init__(self, params: BaseParams, problem_name: str, input_embeddings: tf.keras.layers.Layer=None, share_embedding=True) -> None:
        super(MaskLM, self).__init__(name=problem_name)
        self.params = params
        self.problem_name = problem_name

        if share_embedding is False:
            self.vocab_size = self.params.bert_config.vocab_size
            self.share_embedding = False
        else:
            self.vocab_size = input_embeddings.shape[0]
            embedding_size = input_embeddings.shape[-1]
            share_valid = (self.params.bert_config.hidden_size ==
                        embedding_size)
            if not share_valid and self.params.share_embedding:
                logger.warning(
                    'Share embedding is enabled but hidden_size != embedding_size')
            self.share_embedding = self.params.share_embedding & share_valid

        if self.share_embedding:
            self.share_embedding_layer = TFSharedEmbeddings(
                vocab_size=self.vocab_size, hidden_size=input_embeddings.shape[1])
            self.share_embedding_layer.build([1])
            self.share_embedding_layer.weight = input_embeddings
        else:
            self.share_embedding_layer = tf.keras.layers.Dense(self.vocab_size)

    def call(self, inputs):
        mode = get_phase()
        features, hidden_features = inputs

        # masking is done inside the model
        seq_hidden_feature = hidden_features['seq']
        if mode != PREDICT:
            positions = features['masked_lm_positions']

            # gather_indexes will flatten the seq hidden_states, we need to reshape
            # back to 3d tensor
            input_tensor = gather_indexes(seq_hidden_feature, positions)
            shape_tensor = tf.shape(positions)
            shape_list = tf.concat([shape_tensor, [seq_hidden_feature.shape.as_list()[-1]]], axis=0)
            input_tensor = tf.reshape(input_tensor, shape=shape_list)
            # set_shape to determin rank
            input_tensor.set_shape(
                [None, None, seq_hidden_feature.shape.as_list()[-1]])
        else:
            input_tensor = seq_hidden_feature
        if self.share_embedding:
            mlm_logits = self.share_embedding_layer(
                input_tensor, mode='linear')
        else:
            mlm_logits = self.share_embedding_layer(input_tensor)
        if mode != PREDICT:
            mlm_labels = features['masked_lm_ids']
            mlm_labels.set_shape([None, None])
            mlm_labels = pad_to_shape(from_tensor=mlm_labels, to_tensor=mlm_logits, axis=1)
            # compute loss
            mlm_loss = empty_tensor_handling_loss(
                mlm_labels,
                mlm_logits,
                tf.keras.losses.sparse_categorical_crossentropy
            )
            loss = nan_loss_handling(mlm_loss)
            self.add_loss(loss)

        return tf.nn.softmax(mlm_logits)

## Get or make label encoder function


In [ ]:
# export
def masklm_get_or_make_label_encoder_fn(params: BaseParams, problem: str, mode: str, label_list: List[str], *args, **kwargs) -> LabelEncoder:

    le_path = get_label_encoder_save_path(params=params, problem=problem)

    if need_make_label_encoder(mode=mode, le_path=le_path, overwrite=kwargs['overwrite']):
        # fit and save label encoder
        label_encoder = load_transformer_tokenizer(params.transformer_tokenizer_name, params.transformer_tokenizer_loading)
        pickle.dump(label_encoder, open(le_path, 'wb'))
        try:
            params.set_problem_info(problem=problem, info_name='num_classes', info=len(label_encoder.vocab))
        except AttributeError:
            # models like xlnet's vocab size can only be retrieved from config instead of tokenizer
            params.set_problem_info(problem=problem, info_name='num_classes', info=params.bert_config.vocab_size)
    else:
        # models like xlnet's vocab size can only be retrieved from config instead of tokenizer
        params.set_problem_info(problem=problem, info_name='num_classes', info=params.bert_config.vocab_size)
        label_encoder = pickle.load(open(le_path, 'rb'))

    return label_encoder

## Label handing function

In [ ]:
# export
def masklm_label_handling_fn(target, label_encoder=None, tokenizer=None, decoding_length=None, *args, **kwargs):
    # masklm is a special case since it modifies inputs
    # for more standard implementation of masklm, please see premask_mlm
    return None, None

